In [1]:
import python.dbn_inference as infer

# Find All Possible Inference Variables and Levels

In [2]:
# Inspect available variables from the model
VE = infer.load_DBN_VE_engine()
model = VE.model  # The underlying Bayesian model used by the inference engine

# List all variable names
print("Available variables in the model:")
print(sorted(model.nodes()))

Available variables in the model:
['Block_away', 'Block_away_lag1', 'Block_away_lag2', 'Block_home', 'Block_home_lag1', 'Block_home_lag2', 'DReb_Rate_away', 'DReb_Rate_away_lag1', 'DReb_Rate_away_lag2', 'DReb_Rate_home', 'DReb_Rate_home_lag1', 'DReb_Rate_home_lag2', 'E_OFF_RATING_PERCENTILE_BUCKET_away', 'E_OFF_RATING_PERCENTILE_BUCKET_away_lag1', 'E_OFF_RATING_PERCENTILE_BUCKET_away_lag2', 'E_OFF_RATING_PERCENTILE_BUCKET_home', 'E_OFF_RATING_PERCENTILE_BUCKET_home_lag1', 'E_OFF_RATING_PERCENTILE_BUCKET_home_lag2', 'Ejection_away', 'Ejection_away_lag1', 'Ejection_away_lag2', 'Ejection_home', 'Ejection_home_lag1', 'Ejection_home_lag2', 'EndQuarter', 'EndQuarter_lag1', 'EndQuarter_lag2', 'EndQuarter_lag3', 'FT_Attmpt_PerP_away', 'FT_Attmpt_PerP_away_lag1', 'FT_Attmpt_PerP_away_lag2', 'FT_Attmpt_PerP_home', 'FT_Attmpt_PerP_home_lag1', 'FT_Attmpt_PerP_home_lag2', 'FT_Efficiency_away', 'FT_Efficiency_away_lag1', 'FT_Efficiency_away_lag2', 'FT_Efficiency_home', 'FT_Efficiency_home_lag1', 'FT

In [23]:
# Extract variable levels from CPDs
variable_levels = {
    cpd.variable: cpd.state_names[cpd.variable]
    for cpd in model.cpds
        if cpd.state_names and cpd.variable in cpd.state_names
}

# Print the possible levels for a few example variables
example_vars = ['X2pt_Rate_away', 'X2pt_Rate_home', 'home_momentum0.05', 'away_momentum0.05']
print("\nLevels (states) for selected variables:")
for var in example_vars:
    if var in variable_levels:
        print(f"{var}: {variable_levels[var]}")
    else:
        print(f"{var}: not found in the model.")


Levels (states) for selected variables:
X2pt_Rate_away: [0, 1, 2, 3, 4]
X2pt_Rate_home: [0, 1, 2, 3, 4]
home_momentum0.05: [0, 1, 2]
away_momentum0.05: [0, 1, 2]


# Probabilistic Inference Example

In [6]:
# Load the probabilistic variable elimination inference engine
VE = infer.load_DBN_VE_engine()

In [7]:
# Define a list of evidence scenarios to evaluate.
# Each dictionary corresponds to a different evidence configuration.

evidence_scenarios = [
    {"X2pt_Rate_away": 0},           # Scenario 1: condition on a single value (exact match)
    {"X2pt_Rate_away": [1, 2]},      # Scenario 2: OR condition — treat levels 1 or 2 as possible values
    {"X2pt_Rate_away": 3, "X2pt_Rate_home": 0}  # Scenario 3: multiple exact conditions on two variables
]

In [8]:
# Run probabilistic inference across all evidence scenarios.
# This returns the probability distribution for each momentum variable for each observed evidence scenario.
results = infer.batchProbabilisticInference(VE,
                                            ['home_momentum0.05','away_momentum0.05'], # home and away momentum events with 0.05 explosivness threshold
                                            evidence_scenarios,
                                            elimination_order='greedy',
                                            joint=False)

In [9]:
# Interpretation of momentum variables:
# For both 'home_momentum0.05' and 'away_momentum0.05':
# - Level 0 indicates that no momentum event occurred.
# - Level 1 corresponds to a momentum event whose duration falls between 0 and the 90th percentile.
# - Level 2 indicates a momentum event in the top 10% of duration
#
# Recommended practice: To assess the overall probability of a momentum event occurring, sum the probabilities of levels 1 and 2.
results

,target_node,target_level,target_probability,evidence
0,home_momentum0.05,0,0.779474,{'X2pt_Rate_away': 0}
1,home_momentum0.05,1,0.190994,{'X2pt_Rate_away': 0}
2,home_momentum0.05,2,0.029531,{'X2pt_Rate_away': 0}
3,away_momentum0.05,0,0.834467,{'X2pt_Rate_away': 0}
4,away_momentum0.05,1,0.147223,{'X2pt_Rate_away': 0}
5,away_momentum0.05,2,0.018310,{'X2pt_Rate_away': 0}
6,home_momentum0.05,0,0.800777,"{'X2pt_Rate_away': [1, 2]}"
7,home_momentum0.05,1,0.175483,"{'X2pt_Rate_away': [1, 2]}"
8,home_momentum0.05,2,0.023740,"{'X2pt_Rate_away': [1, 2]}"
9,away_momentum0.05,0,0.831985,"{'X2pt_Rate_away': [1, 2]}"


# Causal Inference Example

In [3]:
# Define intervention scenarios for causal inference.
# Each dictionary contains:
# - intervention_dict: variables to intervene on and the levels to fix them at.
# - evidence_dict: optional observational evidence to condition on, holding certain variables fixed.
#
# Example:
# - First scenario: Intervene to set 'X2pt_Efficiency_home' to level 1, with no additional evidence.
# - Second scenario: Same intervention, but conditioned on observed evidence 'X2pt_Efficiency_away' being level 2.
causal_inference_interventions =  [
    {"intervention_dict": {'X2pt_Efficiency_home': 1}, "evidence_dict": None},
    {"intervention_dict": {'X2pt_Efficiency_home': 1}, "evidence_dict": {'X2pt_Efficiency_away': 2}}
    ]

In [4]:
causal_results = infer.batchCausalInferenceAllDAGs( ['home_momentum0.05','away_momentum0.05'],
                                                    causal_inference_interventions,
                                                    elimination_order='greedy',
                                                    joint=False)

Loaded all 128 DBNs for causal inference.


Loaded all 128 DBNs for causal inference.


In [5]:
# Causal query results for all 128 I-equivalent dags
causal_results

,target_node,target_level,target_probability,evidence,dag_number,intervention
0,home_momentum0.05,0,0.859621,None,50,{'X2pt_Efficiency_home': 1}
1,home_momentum0.05,1,0.128514,None,50,{'X2pt_Efficiency_home': 1}
2,home_momentum0.05,2,0.011865,None,50,{'X2pt_Efficiency_home': 1}
3,away_momentum0.05,0,0.782012,None,50,{'X2pt_Efficiency_home': 1}
4,away_momentum0.05,1,0.191190,None,50,{'X2pt_Efficiency_home': 1}
...,...,...,...,...,...,...
1531,home_momentum0.05,1,0.125514,{'X2pt_Efficiency_away': 2},114,{'X2pt_Efficiency_home': 1}
1532,home_momentum0.05,2,0.010292,{'X2pt_Efficiency_away': 2},114,{'X2pt_Efficiency_home': 1}
1533,away_momentum0.05,0,0.800962,{'X2pt_Efficiency_away': 2},114,{'X2pt_Efficiency_home': 1}
1534,away_momentum0.05,1,0.178060,{'X2pt_Efficiency_away': 2},114,{'X2pt_Efficiency_home': 1}
